# 🌫️ Atmospheric CO₂ Data Cleaning & Preparation

This notebook processes and enriches the **Mauna Loa daily atmospheric CO₂ dataset** from NOAA and Scripps. The goal is to prepare the data for visualization in Tableau and deeper analysis.

---

## 🧹 Data Cleaning Objectives

- Load the raw daily CO₂ data from the NOAA `.txt` file
- Parse and format the date fields correctly
- Remove flagged or invalid entries (e.g., `-99.99` values)
- Handle missing values using appropriate techniques (e.g., interpolation, rolling averages)
- Standardize column names for clarity

---

## 🧪 Feature Engineering Plan

We’ll create a variety of new features to support time-series analysis and storytelling:

| Feature | Description |
|--------|-------------|
| `date`, `year`, `month`, `day`, `day_of_year` | Temporal breakdown |
| `co2_30d_avg`, `co2_365d_avg` | Rolling averages to smooth trends |
| `daily_diff`, `monthly_diff`, `pct_change` | Growth and rate-of-change metrics |
| `anomaly_flag` | Outlier detection based on z-scores or thresholds |
| `sin_day`, `cos_day` | Cyclical encodings for seasonality and radial plots |
| `season` | Categorical: Winter, Spring, Summer, Fall |
| `forecast` | Modeled prediction of future CO₂ levels |

---

## 📦 Output Files

- `co2_with_features.csv` – Cleaned + engineered features for visualization

---

## 🛠️ Tools Used

- `pandas` – Data manipulation
- `numpy` – Numerical operations
- `matplotlib / seaborn` – Data exploration
- `datetime` – Date parsing and manipulation
- `scipy` – Anomaly detection / z-score
- `Prophet` - Forecasting
---

In [4]:
# 📦 Import standard libraries
import pandas as pd
import numpy as np
from io import StringIO
import requests
import os
from skmisc.loess import loess
from scipy.stats import zscore
from prophet import Prophet

# 📊 For quick visual checks
import matplotlib.pyplot as plt
import seaborn as sns


# 🔧 Display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [5]:
# It was quicker to clean up the data in Excel, so we'll skip through any cleaning and load in the csv into df
df = pd.read_csv('../data/co2_daily_cleaned.csv', names = ["year", "month", "day", "decimal", "co2_molfrac"])


In [6]:
# Make a datetime column

df['date'] = pd.to_datetime(df[['year', 'month', 'day']])


In [7]:
# Create day_of_year column

df['day_of_year'] = df['date'].dt.dayofyear


In [8]:
# Setting index to datetime column to calculate 30d/365d rolling averages for co2

df.set_index('date', inplace=True)


In [9]:
# Add rolling 30d/365d rolling averages for co2

df['co2_30d_avg'] = df["co2_molfrac"].rolling(window='30D').mean().round(2)
df['co2_365_avg'] = df["co2_molfrac"].rolling(window='365D').mean().round(2)


In [10]:
# Add daily differences

df['daily_diff'] = df['co2_molfrac'].diff()


In [11]:
# Add monthly differences
monthly = df.resample('M').mean(numeric_only=True)
monthly['monthly_diff'] = monthly['co2_molfrac'].diff().round(2)


In [12]:
# Create daily, and monthly, percentage change

df['pct_change'] = df['co2_molfrac'].pct_change() * 100
monthly['monthly_change'] = monthly['co2_molfrac'].pct_change() * 100


In [13]:
# polar coordinates so we can create seasonal/radial plots

df['sin_day'] = np.sin((2*np.pi*df['day_of_year'])/365) # y value
df['cos_day'] = np.cos((2*np.pi*df['day_of_year'])/365) # x value


In [14]:
# Categorize dates by season
# We'll use meteorlogical seasons since they are a little bit easier to work with

def set_season(month):
    
    if (month == 12 or month == 1 or month == 2):
        return "Winter"
    elif (month == 3 or month == 4 or month == 5):
        return "Spring"
    elif (month == 6 or month == 7 or month == 8):
        return "Summer"
    else:
        return "Autumn"

df['season'] = df.index.to_series().dt.month.apply(set_season)


In [15]:
# Loess smoothing on Day of year and Co2 molfrac

df['co2_loess'] = None

# Loop through each year and apply LOESS
for year, group in df.groupby('year'):
    x = group['day_of_year'].values
    y = group['co2_molfrac'].values

    model = loess(x, y, span=0.2, degree=1)
    model.fit()
    smoothed = model.predict(x).values

    df.loc[group.index, 'co2_loess'] = smoothed

In [16]:
# Detrend the data
yearly_avg = df.groupby('year')['co2_molfrac'].transform('mean')
df['detrended'] = df['co2_molfrac'] - yearly_avg

# Compute seasonal component
seasonal_component = df.groupby('day_of_year')['detrended'].transform('mean')
df['seasonal_component'] = seasonal_component

# Compute deseasoned values and attach annual average to each row
df['deseasoned'] = df['co2_molfrac'] - df['seasonal_component']

# z-score calculation setup for anomaly detection

# calculate within-year means
df['deseasoned_365_avg'] = df.groupby('year')['deseasoned'].transform('mean')

# calculate within-year standard deviation
df['deseasoned_365_std'] = df.groupby('year')['deseasoned'].transform('std')

# calculate 1000 day rolling mean
window = 3650
df['rolling_mean'] = df['deseasoned'].rolling(window=3650, min_periods=1000).mean()

#calculate 1000 day rolling std
df['rolling_std'] = df['deseasoned'].rolling(window=3650, min_periods=1000).std()

# calculate z-score
df['z-score'] = (df['deseasoned'] - df['deseasoned_365_avg'])/df['deseasoned_365_std']

# calculate z-score for 1000 rolling day window

df['z-score_1000'] = (df['deseasoned'] - df['rolling_mean'])/df['rolling_std']

# flag any anomalies, where z-score > 2
df['anomaly_flag'] = (df['z-score']).abs() > 2

# flag any anomalies for 10000 rolling day window

df['anomaly_flag_1000'] = (df['z-score_1000']).abs() > 2

# Create new df to store anomaly data
df_anomaly = pd.DataFrame()
df_anomaly['date'] = df.reset_index().date
df_anomaly = df_anomaly.set_index('date')
df_anomaly[['year', 'month', 'day', 'day_of_year', 'co2_molfrac', 'season', 'detrended', 
            'seasonal_component', 'deseasoned', 'deseasoned_365_avg', 
            'deseasoned_365_std', 'z-score', 'anomaly_flag', 'rolling_mean',
           'rolling_std', 'z-score_1000', 'anomaly_flag_1000']] = df[['year', 'month', 'day', 'day_of_year', 'co2_molfrac', 'season', 'detrended', 
            'seasonal_component', 'deseasoned', 'deseasoned_365_avg', 
            'deseasoned_365_std', 'z-score', 'anomaly_flag', 'rolling_mean',
           'rolling_std', 'z-score_1000', 'anomaly_flag_1000']]

# Remove anomaly-related columns from main df
df = df.drop(columns=['detrended', 'deseasoned', 'seasonal_component', 'deseasoned_365_avg', 
                      'deseasoned_365_std', 'z-score', 'anomaly_flag', 'rolling_mean',
                      'rolling_std', 'z-score_1000', 'anomaly_flag_1000'])

In [37]:
# Create anomaly tooltip dataframe

df_anomaly_tooltips = pd.DataFrame()
df_anomaly_tooltips = df_anomaly[df_anomaly['z-score_1000'].abs() > 2]
df_anomaly_tooltips = df_anomaly_tooltips[['deseasoned', 'rolling_mean', 'z-score_1000']]


In [41]:
# Create explanation column for anomalies
    
def generate_explanation_and_event(row):
    z = row['z-score_1000']
    year = row.name.year
    month = row.name.month

    if z > 2:
        if year == 1998 and month in [3, 4, 5]:
            return "Elevated CO₂ likely due to 1997–98 El Niño and widespread biomass burning.", True
        elif year == 1983 and month in [1, 2, 3]:
            return "CO₂ spike likely influenced by 1982–83 El Niño and El Chichón volcanic effects.", True
        elif year == 2016 and month in [3, 4, 5]:
            return "2015–16 El Niño led to warmer conditions and reduced biospheric CO₂ uptake.", True
        elif year == 2020 and month in [4, 5, 6]:
            return "Atmospheric changes during COVID-19 may have affected CO₂ transport and uptake.", True
        elif year == 2005 and month in [7, 8]:
            return "Amazon drought in 2005 likely reduced rainforest carbon absorption.", True
        elif year == 2010 and month in [7, 8, 9]:
            return "Russian wildfires and heatwave likely contributed to elevated CO₂.", True
        elif year == 2003 and month in [6, 7, 8]:
            return "European heatwave in 2003 may have decreased photosynthetic CO₂ uptake.", True
        else:
            return "Higher-than-expected CO₂; possibly due to warm conditions, drought, or biospheric stress.", False

    return "", False  # For z <= 2


df_anomaly_tooltips[['explanation', 'event']] = df_anomaly_tooltips.apply(generate_explanation_and_event, axis=1, result_type='expand')

    

In [19]:
# co2 forecasting

df_prophet = pd.DataFrame({
    'ds': df.index,
    'y': df['co2_molfrac'].values
})

model = Prophet(yearly_seasonality=True)
model.fit(df_prophet)

# Create future dates (e.g., next 10 years = 120 months)
future = model.make_future_dataframe(periods=5475, freq='D')

# Predict future values
forecast = model.predict(future)


17:42:32 - cmdstanpy - INFO - Chain [1] start processing
17:42:37 - cmdstanpy - INFO - Chain [1] done processing


In [20]:
# Filter out dates so we have past 20 years and future 15 years of data
last_date = df_prophet['ds'].max() - pd.DateOffset(years=25)
forecast_future = forecast[forecast['ds'] > last_date]


In [21]:
# Set index to date
forecast_future = forecast_future.set_index('ds')


In [22]:
# Calculating average yearly co2, percentage change, then loess smoothed curve -- all in a new dataframe

# Step 1: Calculate yearly CO₂ average
yearly_df = df.groupby('year')['co2_molfrac'].mean().reset_index()
yearly_df.rename(columns={'co2_molfrac': 'yearly_co2_avg'}, inplace=True)

# Step 2: Calculate percentage change year over year
yearly_df['pct_change'] = yearly_df['yearly_co2_avg'].pct_change() * 100

# Step 3: Set 'year' as the index
yearly_df.set_index('year', inplace=True)

# Step 4: Apply LOESS smoothing to percentage change
loess_model = loess(
    yearly_df.index.values,
    yearly_df['pct_change'].values,
    span=0.2,
    degree=1
)
loess_model.fit()
smoothed = loess_model.predict(yearly_df.index.values).values

# Optional: Add smoothed values to the DataFrame
yearly_df['smoothed_pct_change'] = smoothed


In [42]:
# Saving the dataframes we created
monthly.to_csv('../data/monthly_features.csv', index=True)
df.to_csv('../data/features.csv', index=True)
forecast_future.to_csv('../data/forecast_future.csv', index=True)
yearly_df.to_csv('../data/yearly_co2_average.csv', index=True)
df_anomaly.to_csv('../data/anomaly.csv', index=True)
df_anomaly_tooltips.to_csv('../data/anomaly_tooltips.csv', index=True)